In [3]:
import pandas as pd
import mxnet
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

#=========================================================#

##나중에 argpaser로 변경
max_grad_norm = 1
log_interval = 100
warmup_ratio = 0.1
num_epochs = 2
batch_size = 4
max_len = 512
learning_rate = 5e-5 

#########################



device = torch.device("cuda:1")
print(f"Using {device}")


data = pd.read_pickle("../result/sample5_tokenized.pkl")
##############
data = data.sample(frac = 0.1)

print(data.info())

label_to_int = {}
for i, item in enumerate(data['접수기관'].unique()):
    label_to_int[item] = i

data['접수기관'] = data['접수기관'].apply(lambda x : label_to_int[x])
data = data[['token', '접수기관']]



dataset_train, dataest_test = train_test_split(data, test_size=0.1, random_state=42)


print(f"train len : {len(dataset_train)}, test len : {len(dataest_test)}")



class BERTDataset(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(dataset.iloc[i]['token'])]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['접수기관']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



print('get bertmodel and vocab')
bertmodel, vocab = get_pytorch_kobert_model()


print("data setting")
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

train_data = BERTDataset(dataset_train, tok, max_len, True, False)
test_data = BERTDataset(dataest_test, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(
    train_data, batch_size = batch_size, num_workers = 8)

test_dataloader = torch.utils.data.DataLoader(
    test_data, batch_size = batch_size, num_workers = 8)


class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = len(dataset_train['접수기관'].unique()),   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


    #BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader


print("Train Start")
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} / {} loss {} train acc {}".format(e+1, batch_id+1 , len(train_dataloader), loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))



Using cuda:1
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 597127 to 343594
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   제목      100000 non-null  object
 1   본문      99979 non-null   object
 2   민원발생지   100000 non-null  object
 3   접수기관    100000 non-null  object
 4   token   100000 non-null  object
dtypes: object(5)
memory usage: 4.6+ MB
None
train len : 90000, test len : 10000
get bertmodel and vocab
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
data setting
using cached model. /home/mglee/VSCODE/git_folder/complain_department_classification/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Train Start


  0%|          | 0/22500 [00:00<?, ?it/s]

epoch 1 batch id 1 / 22500 loss 5.956815719604492 train acc 0.0
epoch 1 batch id 101 / 22500 loss 5.856468200683594 train acc 0.0024752475247524753
epoch 1 batch id 201 / 22500 loss 5.930868625640869 train acc 0.004975124378109453
epoch 1 batch id 301 / 22500 loss 5.782311916351318 train acc 0.01910299003322259
epoch 1 batch id 401 / 22500 loss 5.366364479064941 train acc 0.038029925187032416
epoch 1 batch id 501 / 22500 loss 5.004795074462891 train acc 0.05538922155688623
epoch 1 batch id 601 / 22500 loss 4.809051513671875 train acc 0.06364392678868552
epoch 1 batch id 701 / 22500 loss 4.071645736694336 train acc 0.07346647646219687
epoch 1 batch id 801 / 22500 loss 4.392438888549805 train acc 0.08114856429463171
epoch 1 batch id 901 / 22500 loss 5.835740089416504 train acc 0.08546059933407325
epoch 1 batch id 1001 / 22500 loss 4.386695861816406 train acc 0.09090909090909091
epoch 1 batch id 1101 / 22500 loss 4.859184265136719 train acc 0.09990917347865577
epoch 1 batch id 1201 / 2250

  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 test acc 0.7017


  0%|          | 0/22500 [00:00<?, ?it/s]

epoch 2 batch id 1 / 22500 loss 0.9183655977249146 train acc 0.75
epoch 2 batch id 101 / 22500 loss 1.177146077156067 train acc 0.7054455445544554
epoch 2 batch id 201 / 22500 loss 2.0242714881896973 train acc 0.7251243781094527
epoch 2 batch id 301 / 22500 loss 0.2234921157360077 train acc 0.717607973421927
epoch 2 batch id 401 / 22500 loss 0.35814037919044495 train acc 0.7281795511221946
epoch 2 batch id 501 / 22500 loss 0.1813473105430603 train acc 0.7250499001996008
epoch 2 batch id 601 / 22500 loss 1.1729357242584229 train acc 0.7200499168053245
epoch 2 batch id 701 / 22500 loss 1.8668217658996582 train acc 0.7161198288159771
epoch 2 batch id 801 / 22500 loss 1.2432953119277954 train acc 0.7144194756554307
epoch 2 batch id 901 / 22500 loss 3.2532544136047363 train acc 0.7089345172031076
epoch 2 batch id 1001 / 22500 loss 2.0873913764953613 train acc 0.7107892107892108
epoch 2 batch id 1101 / 22500 loss 1.4951894283294678 train acc 0.7086739327883742
epoch 2 batch id 1201 / 22500 l

In [ ]:
PATH = '../result/kobert/' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'KoBERT_0621.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'Kobert_0621_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar') 